In [2]:
#获取训练集
import geopandas as gpd
import rasterio as rio
from shapely.geometry import mapping
import numpy as np
from rasterio.mask import mask

In [ ]:
#获取训练集
import geopandas as gpd
import rasterio as rio
from shapely.geometry import mapping
import numpy as np
from rasterio.mask import mask


def get_train_data(in_raster, in_shp, attr='Id'):
    """获取训练数据集及其标签"""
    shp_df = gpd.GeoDataFrame.from_file(in_shp)
    with rio.open(in_raster) as src:
        band_count = src.count
        X = np.array([], dtype=np.int16).reshape(band_count, 0)
        y = np.array([], dtype=np.int)
        for index, row in shp_df.iterrows():
            geo_feature = [mapping(row['geometry'])]
            clip_raster = mask(src, geo_feature, crop=True)[0]  # 提取roi
            clip_raster_nozero = clip_raster[:, ~np.all(clip_raster == 0, axis=0)]  # 提取非0像元
            #
            y = np.append(y, [row[attr]] * clip_raster_nozero.shape[1])
            X = np.hstack((X, clip_raster_nozero))
    return X, y